# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [2]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import seaborn
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import collections
import string
import unidecode # to remove accents
%matplotlib inline

In [3]:
# Make a request
url_main = 'https://www.topuniversities.com'  # Found with postman
r = requests.get(url_main + '/sites/default/files/qs-rankings-data/357051.txt')
print('Response status code: {0}\n'.format(r.status_code))
page_body = r.text

# Serialize the json data with json library
rank_json = json.loads(page_body)

Response status code: 200



In [4]:
#build the dataframe
rank_df = pd.DataFrame()
rank_df = rank_df.from_dict(rank_json['data']).head(200)
rank_df.stars
rank_df.drop(['logo', 'stars', 'nid','cc', 'score'], axis=1, inplace=True)
rank_df.set_index('core_id', inplace=True)
rank_df = rank_df[['title', 'rank_display', 'country', 'region', 'url']]

In [5]:
def my_find(html_attributes, new_df_column_name, rank_df):
    _tag = html_attributes['tag']
    _class = html_attributes['class']
    # _list is a temporary list that will store the values found and then be converted in the df to be returned.
    _list = []
    for url in rank_df.url:
        # for every url contained in rank_df['url'], perform the corresponding request:
        uni_url = requests.get(url_main + url)
        uni_body = uni_url.text
        soup = BeautifulSoup(uni_body, 'html.parser')
        # look for <tag=_tag, class=_class>
        soup1 = soup.find(_tag, class_=_class)
        # if such tag has been found, look then for <tag=_tag, class='number> where the value 
        # we're interested in is stored! otherwise append -99
        if soup1:
            soup2 = soup1.find(_tag, class_='number') 
            # if such tag has been found, append its value to the _list, otherwise append -99
            if soup2:
                _list.append({new_df_column_name: soup2.text})
            else:
                _list.append({new_df_column_name: -99})
        else:
            _list.append({new_df_column_name: -99})
    # convert _list to dataframe and return it
    return pd.DataFrame.from_dict(_list).replace({r'\n': ''},\
                                                 regex=True).replace({r',': ''},\
                                                                     regex=True).apply(pd.to_numeric).astype(int)

In [6]:
# defining HTML tag and class attributes that we want to find
tofind = [{'tag':'div', 'class': 'total faculty'}, 
          {'tag':'div', 'class': 'inter faculty'}, 
          {'tag':'div', 'class': 'total student'}, 
          {'tag':'div', 'class': 'total inter'}]

# creating DataFrame with the data found (NaN values = -99)
details_df = pd.concat([my_find(tofind[0], 'fac_memb_tot', rank_df),
                        my_find(tofind[1], 'fac_memb_int', rank_df),
                        my_find(tofind[2], 'nb_stud_tot', rank_df),
                        my_find(tofind[3], 'nb_stud_int', rank_df)], axis=1)

# concatenate the DataFrames into a unique one
details_df.set_index(rank_df.index, inplace=True)
QS_df = pd.concat([rank_df, details_df], axis=1)

# cleaning the unique DataFrame (deleting the = in rank_display)
QS_df.drop(['url'], axis=1, inplace=True)
QS_df.rank_display = QS_df.rank_display.replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# Creating the faculty_members_ratio and number_of_students ratio
QS_df['fac_memb_ratio'] = QS_df.fac_memb_tot / QS_df.nb_stud_tot
QS_df['int_stud_ratio'] = QS_df.nb_stud_int / QS_df.nb_stud_tot

# Deleting what's useless
del details_df, tofind, rank_df, r, page_body, rank_json

In [7]:
# resetting the index, dropping the 'core_id' column wich is useless, and renaming the columns
# in order to prepare themerge with the corresponding TimesHigherEducation dataframe
QS_df.reset_index(inplace = True)
QS_df.drop('core_id', axis=1, inplace = True)
QS_df.columns = ['QS_name', 'QS_rank', 'country', 'region', 'QS_fac_memb_tot', 'QS_fac_memb_int',
                'QS_nb_stud_tot', 'QS_nb_stud_int', 'QS_fac_memb_ratio', 'QS_int_stud_ratio']

# TIMES HIGHER EDUCATION

In [8]:
# Making the request, beautifully soupping for extracting the dataframe
URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/'\
                +'world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL)
print('r = {r} // status_code = {status}'.format(r=r,status=r.status_code))
r.content
soupp = BeautifulSoup(r.content,'html.parser')
rank_json = json.loads(r.text)
THE_df = pd.DataFrame()
THE_df = THE_df.from_dict(rank_json['data']).head(200)

r = <Response [200]> // status_code = 200


In [9]:
# select columns of our interest and changing their name
THE_df = THE_df[['name', 'aliases', 'location', 'rank', 'stats_number_students', 'stats_student_staff_ratio', 'stats_pc_intl_students']]
THE_df.columns=['THE_name', 'THE_aliases', 'THE_location', 'THE_rank','THE_nb_stud_tot', 'THE_stats_student_staff_ratio','THE_int_stud_ratio']

# still the usual, hard-to-understand cleaning of the dataset

In [10]:
# THE_int_stud_ratio
THE_df['THE_int_stud_ratio'] = THE_df['THE_int_stud_ratio'].str.replace('%', '').astype('double')/100

# THE_nb_stud_tot
THE_df['THE_nb_stud_tot'] = THE_df['THE_nb_stud_tot'].str.replace(',', '').astype('int')

# THE_fac_memb_int is a missing data, I add it just to state it clearly
THE_df['THE_fac_memb_int'] = -99

# THE_fac_memb_tot
THE_df['THE_stats_student_staff_ratio']=THE_df['THE_stats_student_staff_ratio'].astype(float)
THE_df['THE_fac_memb_tot'] = THE_df['THE_nb_stud_tot']/THE_df['THE_stats_student_staff_ratio']
THE_df['THE_fac_memb_tot'] = THE_df['THE_fac_memb_tot'].astype(int)

# THE_nb_stud_int
THE_df['THE_nb_stud_int'] = THE_df['THE_nb_stud_tot']*THE_df['THE_int_stud_ratio']
THE_df['THE_nb_stud_int'] = THE_df['THE_nb_stud_int'].astype(int)

# THE_fac_memb_ratio
THE_df['THE_fac_memb_ratio'] = 1/THE_df['THE_stats_student_staff_ratio']
THE_df=THE_df.drop('THE_stats_student_staff_ratio', axis=1)

# THE_rank
THE_df['THE_rank'] = THE_df['THE_rank'].astype(str)
THE_df['THE_rank'] = THE_df['THE_rank'].replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# let's now have a look at the clean THE_df

# MY FUNCTIONS --------------------------------------------------------

In [11]:
# return dataframes df1,df2 with only the rows where index in column1 exist also in column2 and vice versa
# and also a third dataframe with the non mergeable rows.

def exclude_differences(df1,df2,column1,column2):
    tmp1 = df1[column1].value_counts().index
    tmp2 = df2[column2].value_counts().index
    my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
    out_1 = df1.copy()
    out_2 = df2.copy()
    out_3 = out_1.iloc[0:0,:].copy()
    
    for loc in my_diff:
        # we wanna keep track of non meargeable elements
        out_31 = out_1[out_1[column1] == loc] 
        out_32 = out_2[out_2[column2] == loc]
        out_3 = pd.concat([out_3, out_31])
        out_3 = pd.concat([out_3, out_32])
        # now we get rid of these rows
        out_1 = out_1[out_1[column1] != loc] 
        out_2 = out_2[out_2[column2] != loc]
                
    return out_1, out_2, out_3

In [12]:
def merging_by_name(THE,QS,prob_limit):
    # creating two lists with the names of the universities from the two datasets
    the = THE.copy()
    qs = QS.copy()
    THE_name = list(the.loc[:,'THE_name'])
    QS_name = list(qs.loc[:,'QS_name'])

    #initializing a new column of the THE_df with the corresponding QS name found by the matching function
    # just to control that everything went smoothly
    the['THE_corresponding QS name'] = 'unknown'
    the['prob'] = 'unknown'

    # MATCHING FUNCTION
    # finding the probable corresponding name in the QS dataframe for each university
    for i,THE_uni in enumerate(THE_name):
        QS_uni, prob=process.extractOne(THE_uni, QS_name, scorer=fuzz.token_sort_ratio)
        if prob>=prob_limit: #if prob<87, I observed that the algorithm matches diffeent universities!! 97 is a good limit
            the.loc[(the['THE_name']== THE_uni) , "THE_corresponding QS name"] = QS_uni
            the.loc[(the['THE_name']== THE_uni) , 'prob'] = prob
                     
    # MERGING        
    Unique_df=pd.merge(the,qs, left_on='THE_corresponding QS name', right_on='QS_name', how = 'outer')
    return Unique_df

In [13]:
def create_unique_df(THE,QS,prob):
    unique_list = []
    for loc in QS['country'].value_counts().sort_index().index:
        the= THE.loc[THE['THE_location'] == loc , :].copy()
        qs  = QS.loc[QS['country'] == loc , :].copy()
        unique = merging_by_name(the,qs,prob)
        unique = unique.dropna(axis=0,how='any')
        unique_list.append(unique)
    unique_out = pd.concat(unique_list)
    return unique_out

In [14]:
def drop_matched_uni(df,name, list_of_matched_uni):
    out = df.copy()
    for x in out[name]:
        if x in list_of_matched_uni:
            out.drop(out[out[name] == x].index[0], inplace=True)
    return out

In [15]:
def append_to_non_mergeable(non_mergeable,THE,QS,errors_THE,errors_QS):
    out = non_mergeable.copy()

    for x in errors_THE:
        out = pd.concat([out,THE.loc[THE['THE_name'] == x]])
    for x in errors_QS:
        out = pd.concat([out,QS.loc[QS['QS_name'] == x]])

    return out

#### The next two functions are for text transformation improve fuzz accuracy, we first provide examples of how the libraries work:

fuzz influenced by:  äàö _ 

fuzz not influenced by:  éè'-()

In [20]:
# example of use for the library unidecode

accented_string = 'Málaga'
# accented_string is of type 'unicode'
unaccented_string = unidecode.unidecode(accented_string)
# unaccented_string contains 'Malaga'and is of type 'str'
print(unidecode.unidecode('äèàäéöû_èüÈ'))
print(unidecode.unidecode(accented_string))


aeaaeou_euE
Malaga


In [21]:
# example of use for the collections library

table = collections.defaultdict(lambda: None)
table.update({
    ord('é'):'e',
    ord('ô'):'o',
    ord('ö'):'o',
    ord(' '):' ',
    ord('\''):' ',
    ord('-'): ' ',
    ord('\N{NO-BREAK SPACE}'): ' ',
    ord('\N{EN SPACE}'): ' ',
    ord('\N{EM SPACE}'): ' ',
    ord('\N{THREE-PER-EM SPACE}'): ' ',
    ord('\N{FOUR-PER-EM SPACE}'): ' ',
    ord('\N{SIX-PER-EM SPACE}'): ' ',
    ord('\N{FIGURE SPACE}'): ' ',
    ord('\N{PUNCTUATION SPACE}'): ' ',
    ord('\N{THIN SPACE}'): ' ',
    ord('\N{HAIR SPACE}'): ' ',
    ord('\N{ZERO WIDTH SPACE}'): ' ',
    ord('\N{NARROW NO-BREAK SPACE}'): ' ',
    ord('\N{MEDIUM MATHEMATICAL SPACE}'): ' ',
    ord('\N{IDEOGRAPHIC SPACE}'): ' ',
    ord('\N{IDEOGRAPHIC HALF FILL SPACE}'): ' ',
    ord('\N{ZERO WIDTH NO-BREAK SPACE}'): ' ',
    ord('\N{TAG SPACE}'): ' ',
    })
table.update(dict(zip(map(ord,string.ascii_uppercase), string.ascii_lowercase)))
table.update(dict(zip(map(ord,string.ascii_lowercase), string.ascii_lowercase)))
table.update(dict(zip(map(ord,string.digits), string.digits)))

print('123 fôé BAR҉'.translate(table,))
print('Paris-Sorbonne University – Paris 4'.translate(table,))
print('éöàäèüÒàèò'.translate(table,))


123 foe bar
paris sorbonne university  paris 4
eo


In [22]:
def remove_accent_in_name(df,name):
    out=df#.copy()
    for x in out[name]:
        x = unidecode.unidecode(x)
    return out

In [23]:
def remove_characters_in_name(df,name):
    out=df.copy()
    for x in out[name]:
        x = x.translate(table,)
    return out


# Those where our dataset, from now we try to merge

It appears that there is not the same set of countries in the two ranking. The universities that come from a country not present in the other dataset are thus considered not-mergeable. Note the we replaced "Russian federation " by "Russia" to have a match, assuming that they are the same. 

In [24]:
# Set russian federation equal russia for comparing countries
THE_df.loc[THE_df['THE_location']=='Russian Federation','THE_location']='Russia'

# find differences in listed countries
tmp1 = QS_df['country'].value_counts().index
tmp2 = THE_df['THE_location'].value_counts().index
my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
print('Here is my set of country present in only one of the rankings:\n',my_diff)

Here is my set of country present in only one of the rankings:
 ['Malaysia', 'Chile', 'Israel', 'Brazil', 'India', 'Argentina', 'Mexico', 'Luxembourg', 'Saudi Arabia']


## STEP 1 remove unis of countries that are not in both rankings

In [25]:
QS_step1, THE_step1, non_mergeable = exclude_differences(QS_df,THE_df,'country','THE_location')

print('QS_df remaining rows: ',QS_step1.shape[0],'    THE_df remaining rows: ',THE_step1.shape[0])
print('Rows in non-mergeable: ',non_mergeable.shape[0])
QS_step1.sort_values('country',inplace=True)
THE_step1.sort_values('THE_location',inplace=True)

QS_df remaining rows:  187     THE_df remaining rows:  199
Rows in non-mergeable:  14


##### Note that non_meargeable contains only a subset of all the non-meargeables rows, we will append rows to it as the analysis continues

## STEP 2 take out 100% fitting unis names

In [26]:
Unique_df = create_unique_df(THE_step1,QS_step1,100)

# list of matched unis by ranking
QS_matched_uni = list(Unique_df['QS_name'])
THE_matched_uni = list(Unique_df['THE_name'])

# update the remaining ranking dataframes
THE_step2 = drop_matched_uni(THE_step1,'THE_name',THE_matched_uni)
QS_step2 = drop_matched_uni(QS_step1,'QS_name', QS_matched_uni)

# print to check that the dataframes have the correct size:
print('merged rows at this step: ',Unique_df.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step1.shape[0],'   THE after: ',THE_step2.shape[0])
print('QS before: ',QS_step1.shape[0],'   QS after: ',QS_step2.shape[0])

merged rows at this step:  107
Non mergeable dataframe #rows:  14
THE before:  199    THE after:  92
QS before:  187    QS after:  80


## STEP 3 go to 90%

In [27]:
THE_step2_tmp = remove_accent_in_name(THE_step2,'THE_name') # try to remove accents
QS_step2_tmp = remove_accent_in_name(QS_step2,'QS_name') # but without succes

Unique_df2 = create_unique_df(THE_step2_tmp,QS_step2_tmp,90) #90 since 100 gave nothing

# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df2['THE_name'])
QS_matched_uni = list(Unique_df2['QS_name'])

# update the remaining ranking dataframes
THE_step3 = drop_matched_uni(THE_step2,'THE_name',THE_matched_uni)
QS_step3 = drop_matched_uni(QS_step2,'QS_name', QS_matched_uni)

# usual cross check
print('merged rows at this step: ',Unique_df2.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step2.shape[0],'   THE after: ',THE_step3.shape[0])
print('QS before: ',QS_step2.shape[0],'   QS after: ',QS_step3.shape[0])

merged rows at this step:  37
Non mergeable dataframe #rows:  14
THE before:  92    THE after:  55
QS before:  80    QS after:  43


## STEP 4 go to 80%

In [28]:
THE_step3_tmp = remove_characters_in_name(THE_step3,'THE_name') # tried to remove special characters
QS_step3_tmp = remove_characters_in_name(QS_step3,'QS_name') # doesn't work though

Unique_df3 = create_unique_df(THE_step3_tmp,QS_step3_tmp,80) 

# errors made
errors = ["Trinity College Dublin",'University College Dublin']

# update the non-mergeable dataframe
non_mergeable = pd.concat([non_mergeable,THE_step3.loc[THE_step3['THE_name'] == "Trinity College Dublin"]]) 
non_mergeable = pd.concat([non_mergeable,QS_step3.loc[QS_step3['QS_name'] == "University College Dublin"]]) 

Unique_df3 = Unique_df3.loc[Unique_df3['THE_name'] != "Trinity College Dublin"] #the only one with an error

# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df3['THE_name'])  + errors
QS_matched_uni = list(Unique_df3['QS_name']) + errors

# update the remaining ranking dataframes
THE_step4 = drop_matched_uni(THE_step3,'THE_name',THE_matched_uni)
QS_step4 = drop_matched_uni(QS_step3,'QS_name', QS_matched_uni)

# remove duplicates ()usefull mainly if we runs multiples times this cell
non_mergeable = non_mergeable.drop_duplicates()

# usual cross check 
print('merged rows at this step: ',Unique_df3.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step3.shape[0],'   THE after: ',THE_step4.shape[0])
print('QS before: ',QS_step3.shape[0],'   QS after: ',QS_step4.shape[0])

merged rows at this step:  8
Non mergeable dataframe #rows:  16
THE before:  55    THE after:  46
QS before:  43    QS after:  34


## STEP 5 go to 70%

In [29]:
# df created with the matching function, but some errors are present
Unique_df4 = create_unique_df(THE_step4,QS_step4,70) 

# errors at this step
errors = ['University of Sussex','University of Reading',\
          'University of East Anglia','University of Bath',\
          'University of Leicester','University of Dundee']

# errors by ranking
errors_THE = ['University of Sussex','University of East Anglia','University of Dundee','University of Leicester']
errors_QS = ['University of Reading','University of Bath']

# clean the unique_df from its errors
for x in Unique_df4['QS_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['QS_name'] != x]
for x in Unique_df4['THE_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['THE_name'] != x]

#update non mergeable df
non_mergeable = append_to_non_mergeable(non_mergeable,THE_step4,QS_step4,\
                                                    errors_THE,errors_QS)
#drop duplicates
non_mergeable = non_mergeable.drop_duplicates()

# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df4['THE_name'])  + errors_THE
QS_matched_uni = list(Unique_df4['QS_name']) + errors_QS

# update the remaining ranking dataframes
THE_step5 = drop_matched_uni(THE_step4,'THE_name',THE_matched_uni)
QS_step5 = drop_matched_uni(QS_step4,'QS_name', QS_matched_uni)

# usual cross check 
print('merged rows at this step: ',Unique_df4.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step4.shape[0],'   THE after: ',THE_step5.shape[0])
print('QS before: ',QS_step4.shape[0],'   QS after: ',QS_step5.shape[0])

merged rows at this step:  3
Non mergeable dataframe #rows:  22
THE before:  46    THE after:  39
QS before:  34    QS after:  29


## Final STEP all.

In [30]:
# create merged dataframe with matching function. But some errors are present
Unique_df5 = create_unique_df(THE_step5,QS_step5,10)

# correct matchings for THE_ranking (since they are unique due to our merging function)
rights = ['University of Tübingen','LMU Munich','University of Freiburg','Scuola Superiore Sant’Anna']

# errors made: (incorrect matching)
errors = list(set(rights).symmetric_difference(set(list(Unique_df5['THE_name']))))

for x in errors:
    Unique_df5 = Unique_df5[Unique_df5['THE_name'] != x] 


# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df5['THE_name']) 
QS_matched_uni = list(Unique_df5['QS_name']) 

# update the remaining ranking dataframes
THE_step6 = drop_matched_uni(THE_step5,'THE_name',THE_matched_uni)
QS_step6 = drop_matched_uni(QS_step5,'QS_name', QS_matched_uni)

# update non_mergeables unis dataframe
non_mergeable = pd.concat([non_mergeable,THE_step6,QS_step6])
non_mergeable = non_mergeable.drop_duplicates()

# usual cross check 
print('merged rows at this step: ',Unique_df5.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step5.shape[0],'   THE after: ',THE_step6.shape[0])
print('QS before: ',QS_step5.shape[0],'   QS after: ',QS_step6.shape[0])

merged rows at this step:  4
Non mergeable dataframe #rows:  82
THE before:  39    THE after:  35
QS before:  29    QS after:  25


## Final check: 

In [41]:
merged_df_ori = pd.concat([Unique_df,Unique_df2,Unique_df3,Unique_df4,Unique_df5])
num_merged = merged_df.shape[0]

non_mergeable = non_mergeable.drop_duplicates()
num_non_merged = non_mergeable.shape[0]
print ('merged: ',num_merged,'  non-merged',num_non_merged,'\nmissing',(200-num_merged)*2-num_non_merged)
print('\nif 0 missing, it is probable the merge is successfull')

merged:  159   non-merged 82 
missing 0

if 0 missing, it is probable the merge is successfull


# FROM NOW IT IS ORIGINAL WORK AGAIN, we can put it after the one of martino

In [73]:
merged_df = merged_df_ori.copy()

merged_df = merged_df[['THE_name', 'THE_rank','QS_rank','country','region',\
                       'THE_nb_stud_tot','QS_nb_stud_tot', 'THE_int_stud_ratio', 'QS_int_stud_ratio',\
                       'THE_fac_memb_int','QS_fac_memb_int','THE_fac_memb_tot','QS_fac_memb_tot',\
                       'THE_nb_stud_int','QS_nb_stud_int', 'THE_fac_memb_ratio','QS_fac_memb_ratio']]

merged_df.columns = [['University', 'THE_rank','QS_rank','country','region',\
                       'THE_nb_stud_tot','QS_nb_stud_tot', 'THE_int_stud_ratio', 'QS_int_stud_ratio',\
                       'THE_fac_memb_int','QS_fac_memb_int','THE_fac_memb_tot','QS_fac_memb_tot',\
                       'THE_nb_stud_int','QS_nb_stud_int', 'THE_fac_memb_ratio','QS_fac_memb_ratio']]

We recall that THE_fac_memb_int has been set to -99 since the values where non-sense. When averaging between the two rankings, we will take only the value from QS.

In [62]:
# the one with all the info
merged_df_final = merged_df.copy()
# the one with averaged info
merged_df_avg = merged_df.copy()

In [67]:
merged_df_avg['nb_stud_tot'] = (merged_df_avg['THE_nb_stud_tot'] + merged_df_avg['QS_nb_stud_tot'])/2
merged_df_avg['int_stud_ratio'] = (merged_df_avg['THE_int_stud_ratio'] + merged_df_avg['QS_int_stud_ratio'])/2
merged_df_avg['fac_memb_int'] = merged_df_avg['QS_fac_memb_int'] # since no fac_member_ratio
merged_df_avg['fac_memb_tot'] = (merged_df_avg['THE_fac_memb_tot'] + merged_df_avg['QS_fac_memb_tot'])/2
merged_df_avg['nb_stud_int'] = (merged_df_avg['THE_nb_stud_int'] + merged_df_avg['QS_nb_stud_int'])/2
merged_df_avg['fac_memb_ratio'] = (merged_df_avg['THE_fac_memb_ratio'] + merged_df_avg['QS_fac_memb_ratio'])/2

In [71]:
merged_df_avg = merged_df_avg[['University', 'THE_rank', 'QS_rank', 'country', 'region',\
                               'nb_stud_tot', 'int_stud_ratio', 'fac_memb_int', 'fac_memb_tot',\
                               'nb_stud_int', 'fac_memb_ratio']].sort_values('country')

In [72]:
merged_df_avg

,University,THE_rank,QS_rank,country,region,nb_stud_tot,int_stud_ratio,fac_memb_int,fac_memb_tot,nb_stud_int,fac_memb_ratio
7,Monash University,80.0,60.0,Australia,Oceania,52139.5,0.319148,1679.0,2297.0,16847.0,0.042730
1,University of Adelaide,134.0,109.0,Australia,Oceania,21240.5,0.321008,633.0,1174.0,6818.0,0.055279
3,University of Queensland,65.0,47.0,Australia,Oceania,38232.5,0.268944,1870.0,2124.5,10275.5,0.056116
5,University of Western Australia,111.0,93.0,Australia,Oceania,18659.0,0.248442,809.0,1140.5,4635.5,0.061250
6,Australian National University,48.0,20.0,Australia,Oceania,15214.0,0.367183,927.0,1214.0,5573.0,0.081301
0,University of New South Wales,85.0,45.0,Australia,Oceania,39958.5,0.349620,1612.0,2213.5,13968.5,0.055475
2,University of Sydney,61.0,50.0,Australia,Oceania,45615.5,0.342420,1829.0,2711.0,15643.0,0.059139
0,University of Melbourne,32.0,41.0,Australia,Oceania,42149.0,0.413717,1477.0,2447.0,17438.0,0.058044
0,University of Vienna,165.0,154.0,Austria,Europe,40671.0,0.287374,1400.0,2555.5,11832.5,0.061141
0,Ghent University,107.0,125.0,Belgium,Europe,35938.5,0.102352,482.0,1760.0,3678.0,0.048959
